In [1]:
import numpy as np
import matplotlib.pyplot as plt
from uncertainties import ufloat
from uncertainties.umath import *
from uncertainties import unumpy as unp
from scipy.optimize import curve_fit
from scipy import stats

### Note - Python uses the equation $ \sigma_f^2 = \sigma_x^2 \Big( \frac{\partial f}{\partial x} \Big) + \sigma_y^2 \Big( \frac{\partial f}{\partial y} \Big) + \sigma_z^2 \Big( \frac{\partial f}{\partial z} \Big) + ...$ in the uncertainties package to propagate the uncertainty 
### u.float(x,y) is the uncertainties package method to include nominal value, x, and std_devs, y, into one number so that it may calculate uncertainty propagations using the above equation. 

### finding isomer shift: 
$$\delta_i = (factor)*AverageChannel \quad ; \quad factor = \frac{velocity}{channel} \quad and \quad AverageChannel=average\ channel\ from\ center\ peak$$
#### AverageChannel is for when dealing with multiple peaks like in Sodium Nitroprusside, however in stainless steel we will use just the channels away from channel 500 

### finding quadrupole spritting:
$$\delta_Q = (factor)*\delta_{channel} \quad ; \quad where \quad \delta_{channel}=difference \ in \ channel \ from \ channel \ 500$$

### From the Write-up:
 $$Transducer Velocity = 0.295\pm0.002 \frac{mm/s}{mV} $$

#### We know, by our set-up: 
$$1channel = 0.5ms $$
$$0channel=MaxSpeed$$
$$500channel = 0speed$$

#### Goal is to find $\large \frac{velocity}{channel}$

In [3]:
transducer_conversion_factor = ufloat(0.295,0.002) #(mm/(s*mV)
#transducer_factor_unc = 0.002 #(mm/(s*mV)

sweep_voltage = ufloat(49.6, 0.8) #(mV)
#v_uncertainty = 0.8 #(mV)

transducer_velocity = -sweep_voltage / 2 * transducer_conversion_factor 
print("transducer velocity = "+str(transducer_velocity)+ "mm/s")

vel_per_channel = transducer_velocity/500 
print("velocity/channel = "+str(vel_per_channel)+ "mm/s")

transducer velocity = -7.32+/-0.13mm/s
velocity/channel = -0.01463+/-0.00026mm/s


# Enriched Iron $Fe^{57}$ - Magnetic splitting

In [30]:
mu_g_over_mu_n = ufloat(0.0903,0) #given in write-up

E1_peak = ufloat(130.91, 1.11) #(channel)
E2_peak = ufloat(286.30, 1.11) #(channel)
E3_peak = ufloat(443.63, 1.11) #(channel)
E4_peak = ufloat(562.69, 1.11) #(channel)
E5_peak = ufloat(720.61, 1.11) #(channel)
E6_peak = ufloat(879.24, 1.11) #(channel)
peak_channels = [E1_peak, E2_peak, E3_peak, E4_peak, E5_peak, E6_peak]

E_values = [] #will be in the form [E1, E2, E3, E4, E5, E6]
for i in range(len(peak_channels)):
    E_values.append((peak_channels[i] - 500) * vel_per_channel)
Fe_57_isomer = (np.sum(E_values)) / 6
print("Isomer shift for Enriched Iron,\n Delta_i = " +str(Fe_57_isomer)+"mm/s")

delta_g = E_values[1]-E_values[3] #delta_g=E_2-E_4=E_3-E_5 (ground state split energy difference)
print("\nDifference between E(1/2,1/2) and E(1/2,-1/2),\n Delta_g = "+str(delta_g)+"mm/s")

delta_e =  E_values[1]-E_values[2]#delta_e=E1-E2=E2-E3=E3-E4=E4-E5=E5-E6 (energy difference between consecutive levels)
print("\nDifference between consecutive energies,\n Delta_e = "+str(delta_e)+"mm/s")

B = delta_g/(2*mu_g_over_mu_n) #magnetic field as iron nucleus
print("\nMagnetic field at the site of the iron nucleus,\n B = "+str(B)+"T")

mu_e_over_mu_g = (3 * delta_e)/delta_g 
print("\nMagnetic moment of excited state at 14.41eV,\n mu_e/mu_g = "+str(mu_e_over_mu_g))

Isomer shift for Enriched Iron,
 Delta_i = -0.057+/-0.007mm/s

Difference between E(1/2,1/2) and E(1/2,-1/2),
 Delta_g = 4.04+/-0.07mm/s

Difference between consecutive energies,
 Delta_e = 2.30+/-0.05mm/s

Magnetic field at the site of the iron nucleus,
 B = 22.4+/-0.4T

Magnetic moment of excited state at 14.41eV,
 mu_e/mu_g = 1.708+/-0.015


# Stainless Steel - Isomer Shift

In [9]:
channel_away = ufloat(13,0.34)
stainless_isomer_shift = vel_per_channel*channel_away
print("The Isomershift for stainless steel = "+str(np.round(unp.nominal_values(stainless_isomer_shift),4))+
      "+/-"+str(np.round(unp.std_devs(stainless_isomer_shift),4))+ "mm/s")

The Isomershift for stainless steel = -0.1902+/-0.006mm/s


# Sodium Nitroprusside - Quadrupole splitting

In [12]:
left_channel = ufloat(-40,0.71)
right_channel = ufloat(80,0.71)

delta_channel = left_channel-right_channel

average_channel = 1/2 * (left_channel+right_channel)

sodium_isomer_shift = vel_per_channel*average_channel
print("The Isomer shift for Sodium Nitroprusside = "+str(np.round(unp.nominal_values(sodium_isomer_shift),3))+
      "+/-"+str(np.round(unp.std_devs(sodium_isomer_shift),3))+ "mm/s")
sodium_quad_split = vel_per_channel*delta_channel
print("The Quadrupol shift for Sodium Nitroprusside = "+str(np.round(unp.nominal_values(sodium_quad_split),3))+
      "+/-"+str(np.round(unp.std_devs(sodium_quad_split),3))+ "mm/s")

The Isomer shift for Sodium Nitroprusside = -0.293+/-0.009mm/s
The Quadrupol shift for Sodium Nitroprusside = 1.756+/-0.034mm/s
